# Full regression from scratch

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Given data
Y = np.array([
    372,
    206,
    175,
    154,
    136,
    112,
    55,
    45,
    222,
    170
])

X = np.array([
    [1, 46, 162],
    [1, 55, 233],
    [1, 61, 232],
    [1, 66, 232],
    [1, 71, 231],
    [1, 71, 237],
    [1, 81, 224],
    [1, 86, 219],
    [1, 53, 203],
    [1, 60, 188]
])



In [19]:
n =len(Y)
p=X.shape[1]
# Compute the regression coefficients using the normal equation
C = np.linalg.inv(X.T @ X)
B = C @ X.T @ Y
H = X @ C @ X.T
# Compute predicted Y values
Y_pred = X @ B
e = Y- Y_pred

In [4]:
import pandas as pd

actual = pd.DataFrame(Y, columns=["Y"])
prediction = pd.DataFrame(Y_pred, columns=["Y Pred"])
error =pd.DataFrame(e, columns=["e"])
comparison = pd.concat([actual,prediction, error], axis=1)
comparison

,Y,Y Pred,e
0,372,329.528946,42.471054
1,206,208.090335,-2.090335
2,175,173.321416,1.678584
3,154,143.550935,10.449065
4,136,114.736113,21.263887
5,112,109.002163,2.997837
6,55,61.884760,-6.884760
7,45,36.892571,8.107429
8,222,248.668279,-26.668279
9,170,221.324482,-51.324482


In [24]:
variance = (e.T @ e ) / n
print("variance",variance)

variance 583.9830269395452


### Model Evaluation

In [ ]:
# Residual Sum of Squares (RSS)
RSS = np.sum((Y - Y_pred) ** 2)

# Total Sum of Squares (TSS)
TSS = np.sum((Y - np.mean(Y)) ** 2)


# Residual Standard Error (RSE)
RSE = np.sqrt(RSS / (n - p))

# Coefficient of determination
R_square = 1 - RSS/TSS

# F-test GENERAL, TESTING WHOLE MODEL
F = ((TSS - RSS) / (p - 1)) / (RSS / (n - p))

In [28]:
# ---- t-Student Test for the Coefficients ----
from scipy.stats import t

# Variance-covariance matrix of beta estimates:
# Var(B) = RSE^2 * (X^T X)^{-1}
var_beta = RSE**2 * np.linalg.inv(X.T @ X)

# Standard errors for each coefficient (sqrt of diagonal elements):
se_beta = np.sqrt(np.diag(var_beta))

# t-statistic for each coefficient:
# t_i = B_i / SE(B_i)
t_stats = B / se_beta

# Degrees of freedom for residuals:
df = n - p

# Two-tailed p-values for the t-statistics:
p_values = 2 * (1 - t.cdf(np.abs(t_stats), df))

print("\n--- t-Student Test for Coefficients ---")
for i in range(len(B)):
    print(f"B[{i}]: {B[i]:.4f}, SE: {se_beta[i]:.4f}, t: {t_stats[i]:.4f}, p-value: {p_values[i]:.4f}")



--- t-Student Test for Coefficients ---
B[0]: 758.2340, SE: 85.4649, t: 8.8719, p-value: 0.0000
B[1]: -5.9541, SE: 0.9162, t: -6.4988, p-value: 0.0003
B[2]: -0.9557, SE: 0.4688, t: -2.0387, p-value: 0.0809


### Model selection


In [27]:
# Akaike information criterion, smaller the better.
# Also corrected version, improved for small samples

# ---- AIC and AICc ----
# AIC = n [ ln(2π * RSS/n) + 1 ] + 2 (p + 1)
AIC = n * (np.log(2 * np.pi * (RSS / n)) + 1) + 2 * (p + 1)

# AICc = AIC + Correction
# AICc = n [ ln(2π * RSS/n) + 1 ] + 2 (p + 1) * n / (n - p - 1)
# or equivalently:
# AICc = AIC + 2(p+1)(p+2)/(n - p - 1) for typical derivations,
# but we'll directly use the formula given:
AICc = n * (np.log(2 * np.pi * (RSS / n)) + 1) + 2 * (p + 1) * (n / (n - p - 1))

print("\nAIC:", AIC)
print("AICc:", AICc)


AIC: 100.07748985355539
AICc: 105.41082318688872
